In [1]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/nyanko_MLP')

In [5]:
import torch
import numpy
from features.FeatureFunction2 import (
    AttackFeature_solo,AttackFeature_mass,
    DefenseFeature_solo,DefenseFeature_mass,
    DisturbFeature_solo,DisturbFeature_mass,Wave_DisturbFeature,
    CostFeature,AttributeFeature,
    WPOFeature,RoleFeature,
    RangeFeature,HandleFeature,
    GimmickFeature,AtFreFeature
)
from features.FeatureFunctionSpirit2 import(
    AttackFeatureSpirit,
    DefenseFeatureSpirit,
    DisturbFeatureSpirit,
    WPOFeatureSpirit,
)
from model.Spirit_router import SpiritRouter
from normalizer.base import StandardScaler
from model.mlp import MLP,FullModel
from model.feature_concat import FeatureConcat
from data.Rawdata import Rawdata
from model.normalizedFM import NormalizedFeatureModel
from data.SaveManager import SaveManager
from data.NyankoDataset import NyankoDataset
from data.FeatureEncorder import FeatureEncorder,TensorEncorder

# ======================
# device
# ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ======================
# feature 定義（学習時と完全一致）
# ======================
features_normal = [
    AttackFeature_solo(device),
    AttackFeature_mass(device),
    DefenseFeature_solo(device),
    DefenseFeature_mass(device),
    DisturbFeature_solo(device),
    DisturbFeature_mass(device),
    Wave_DisturbFeature(device),
    CostFeature(device),
    AttributeFeature(device),
    WPOFeature(device),
    RoleFeature(device),
    RangeFeature(device),
    HandleFeature(device),
    GimmickFeature(device),
    AtFreFeature(device)
]

features_spirit = [
    AttackFeature_solo(device),
    AttackFeatureSpirit(device),
    DefenseFeature_solo(device),
    DefenseFeatureSpirit(device),
    DisturbFeature_solo(device),
    DisturbFeatureSpirit(device),
    Wave_DisturbFeature(device),
    CostFeature(device),
    AttributeFeature(device),
    WPOFeatureSpirit(device),
    RoleFeature(device),
    RangeFeature(device),
    HandleFeature(device),
    GimmickFeature(device),
    AtFreFeature(device)
]


# ======================
# Dataset（評価用 Excel）
# ※ mu, sigma は一切使わない
# ======================
rawdata = Rawdata("/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/data/nyanko_DB100_dummy.xlsx")
feature_encoder = FeatureEncorder()
tensor_encoder = TensorEncorder(device=device)  # CPU tensorのまま
dataset = NyankoDataset(rawdata, feature_encoder, tensor_encoder, target_col="評価値")

router = SpiritRouter(features_normal,features_spirit)

feature_concat = FeatureConcat(router)
# ======================
# ダミー mu, sigma を作る
# （shape が合っていれば値は何でもいい）
# ======================
sample_row = dataset.rows[0]

with torch.no_grad():
    dummy_feat = feature_concat(sample_row)

dummy_mu = torch.zeros_like(dummy_feat).to(device)
dummy_sigma = torch.ones_like(dummy_feat).to(device)

normalizer = StandardScaler(dummy_mu, dummy_sigma).to(device)

# ======================
# model 構築
# ======================
norm_feature_model = NormalizedFeatureModel(feature_concat, normalizer).to(device)

mlp = MLP(
    input_dim=dummy_feat.shape[0],
    hidden_dim=36
).to(device)

model = FullModel(norm_feature_model, mlp).to(device)

# ======================
# 学習済みモデルをロード
# （ここで mu, sigma が復元される）
# ======================
saver = SaveManager(
    "/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/weight_data"
)


# saver は SaveManager のインスタンス
model, ckpt = saver.load(
    model,
    "/content/drive/MyDrive/ColabNotebooks/nyanko_MLP/weight_data/hide36_e400_l2.6606.pth"
)

# extra_state や epoch も確認できる
if "extra_state" in ckpt:
    print(ckpt["extra_state"])
print(ckpt["epoch"], ckpt["loss"])





feature_mu = ckpt["extra_state"]["feature_mu"].to(dtype=torch.float32,device=device)
feature_sigma = ckpt["extra_state"]["feature_sigma"].to(dtype=torch.float32,device=device)

feature_sigma = torch.where(feature_sigma==0,torch.ones_like(feature_sigma),feature_sigma)

normalizer.mu.copy_(feature_mu)
normalizer.sigma.copy_(feature_sigma)

y_mu = ckpt["extra_state"]["y_mu"].to(dtype=torch.float32,device=device)
y_sigma = ckpt["extra_state"]["y_sigma"].to(dtype=torch.float32,device=device)

def inverse_z(y_z):
    return y_z * y_sigma + y_mu

model.eval()

# ======================
# 推論
# ======================
with torch.no_grad():
    for i in range(len(dataset)):
        row = dataset.rows[i]
        y = dataset.targets[i]

        y_pred_z = model(row)
        y_pred = inverse_z(y_pred_z).item()

        print(i, row.get("name"), f"{y_pred:.2f}", y.item())


device: cpu
{'y_mu': tensor(5.5472), 'y_sigma': tensor(2.4108), 'feature_mu': tensor([1.3623e+01, 1.8116e+01, 1.9639e+02, 5.9958e+01, 8.7928e+00, 5.0178e+00,
        2.5727e+00, 1.5182e-01, 2.6981e+00, 2.8293e-01, 4.5283e-01, 2.3585e-01,
        2.0755e-01, 3.2075e-01, 1.6038e-01, 1.2264e-01, 2.6415e-01, 2.3585e-01,
        8.4103e+00, 3.2310e+01, 1.1321e-01, 0.0000e+00, 3.7736e-02, 9.4340e-03,
        0.0000e+00, 1.8868e-02, 2.3585e-01, 1.1321e-01, 7.5472e-02, 1.1321e-01,
        1.0377e-01, 1.2264e-01, 1.3208e-01, 9.4340e-03, 9.4340e-02, 8.4906e-02,
        3.7736e-02, 9.4340e-03, 2.8302e-02, 3.7736e-02, 5.6604e-02, 5.6604e-02,
        1.0134e+00]), 'feature_sigma': tensor([9.3265e+00, 1.1387e+01, 2.7799e+02, 4.2875e+01, 1.1976e+01, 1.0570e+01,
        2.4106e+01, 7.4278e-02, 3.3790e+00, 5.1473e-02, 5.0013e-01, 4.2655e-01,
        4.0748e-01, 4.6898e-01, 3.6870e-01, 3.2958e-01, 4.4297e-01, 4.2655e-01,
        8.3307e-01, 3.5674e+01, 3.1835e-01, 0.0000e+00, 1.9146e-01, 9.7129e-02,
   